In [45]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score

In [2]:
all_meta = pd.read_csv('data/train_metadata_extended.csv')
with open('data/scored_birds.json') as fin:
    test_birds = json.load(fin)

In [3]:
all_meta.loc[:, 'secondary_labels'] = all_meta.secondary_labels.apply(eval)
all_meta['target_raw'] = all_meta.secondary_labels + all_meta.primary_label.apply(lambda x: [x])

In [4]:
all_species = sorted(set(all_meta.target_raw.sum()))
species2id = {s: i for i, s in enumerate(all_species)}
id2species = {i: s for i, s in enumerate(all_species)}

all_meta['target'] = all_meta.target_raw.apply(lambda species: [int(s in species) for s in all_species])

In [13]:
last_pred = np.load('Lab02_NMT/last_pred.npy')
last_true = np.load('Lab02_NMT/last_true.npy')

In [14]:
test_idx = [species2id[b] for b in test_birds]
last_true[:, test_idx].sum(axis=0)

array([  4.,   6.,  25.,   3.,   0.,   7.,   1.,  13.,   0.,   2.,   1.,
         0.,  87.,  16.,  18.,   0.,   9.,   0., 107.,  22.,  10.],
      dtype=float32)

In [67]:
def score_pred(true, pred):
    if true.sum() == 0:
        return np.nan
    return balanced_accuracy_score(
            true,
            pred
        )

In [71]:
score_stat = []
trsh = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]

for t in trsh:
    score_stat.append({
        b: score_pred(
            last_true[:, species2id[b]],
            last_pred[:, species2id[b]] > t
        )
        for b in test_birds
    })
    
score_stat = pd.DataFrame(score_stat).T
score_stat.columns = trsh

In [72]:
score_stat

,0.01,0.05,0.10,0.15,0.20,0.25
akiapo,0.933434,0.992922,0.998146,0.874663,0.875000,0.875000
aniani,0.907589,0.909415,0.831478,0.832659,0.583165,0.500000
apapan,0.774779,0.971487,0.948798,0.954908,0.896775,0.877794
barpet,0.953167,0.997136,0.832491,0.832996,0.833165,0.833333
crehon,NaN,NaN,NaN,NaN,NaN,NaN
elepai,0.919872,0.921486,0.855625,0.785208,0.714117,0.571429
ercfra,0.969697,0.497980,0.499495,0.499832,0.500000,0.500000
hawama,0.829784,0.902962,0.879037,0.843111,0.805664,0.729417
hawcre,NaN,NaN,NaN,NaN,NaN,NaN
hawgoo,0.968171,0.997979,0.749832,0.500000,0.500000,0.500000


In [75]:
score_stat.idxmax(axis=1).fillna(0.05).to_dict()

{'akiapo': 0.1,
 'aniani': 0.05,
 'apapan': 0.05,
 'barpet': 0.05,
 'crehon': 0.05,
 'elepai': 0.05,
 'ercfra': 0.01,
 'hawama': 0.05,
 'hawcre': 0.05,
 'hawgoo': 0.05,
 'hawhaw': 0.01,
 'hawpet1': 0.05,
 'houfin': 0.1,
 'iiwi': 0.2,
 'jabwar': 0.05,
 'maupar': 0.05,
 'omao': 0.05,
 'puaioh': 0.05,
 'skylar': 0.1,
 'warwhe1': 0.05,
 'yefcan': 0.15}